In [1]:
import os # to check if files exist
import re #to split the text file
import random #to choose questions randomly
import requests #to send the cards to anki
from concurrent.futures import ThreadPoolExecutor #to speed up sending the cards to anki

# Read the content of the question file
file_path = "test your understanding.txt"
with open(file_path, "r") as file:
    content = file.read()

# Split the content at "Test your understanding"
substrings = content.split("Test your understanding")

# Initialize a nested list to store non-empty lines after each substring
nested_list = []

#split before new task
pattern = re.compile(r'\n(?=.{1,2}\))')

# Process each substring and split non-empty lines
for substring in substrings[1:]:  # Start from index 1 to skip the first empty substring
    lines = re.split(pattern, substring)
    nested_list.append(lines[1:]) #first line is also empty

In [2]:
# Read answer files and put them in the answer dict
answer_dict = {}

def split_tut_question_text(string):
        return re.findall(pattern= r'^(\d{1,2})\s(.{1,2}\))\s(.+)', string = string)

def append_to_answer_dict(answer_list, name):
    for line in answer_list:
        answer = split_tut_question_text(string = line)
        tut_num = int(answer[0][0]) 
        question_num = answer[0][1]
        answer_text = f"{name}- " + answer[0][2]
        if tut_num not in answer_dict:
            answer_dict[tut_num] = {question_num:answer_text}
        else: 
            if question_num not in answer_dict[tut_num]:
                answer_dict[tut_num][question_num] = answer_text
            else:
                answer_dict[tut_num][question_num] = answer_dict[tut_num][question_num] + "\n" + answer_text

answer_files = []
for file in os.listdir():
    match = re.match(r"test your understanding answers (.*)\.txt", file)
    if match:
        file_path = os.path.join(os.getcwd(), file)
        with open(file_path, "r") as file_content:
            answer_files.append([file, match.group(1), file_content.read()])

for file in answer_files:
    answers_unsplit = file[2]
    name = file[1]
    
    pattern_answer_lines = re.compile(r'\n(?=\d{1,2} .{1,2}\))')
    if(len(answers_unsplit)>1):
        lines_answers = re.split(pattern_answer_lines, answers_unsplit)
        append_to_answer_dict(lines_answers, name = name)

#function to get question num
def get_question_num(question):
    return  re.findall(pattern = r"^.{1,2}\)", string = question)[0]

#function to find answer
def find_answer(question, tut_num):
    question_num = get_question_num(question)
    answer = "Didn't answer yet"
    if tut_num in answer_dict.keys():
        if question_num in answer_dict[tut_num].keys():
            answer = answer_dict[tut_num][question_num]
    return(answer)


#function test yoursef using a random subset of numbers
def ask_question(question):
    return input(question + '\n')

def test(n = 1, question_type = "unanswered", show_all_questions_first = False, show_answers = True, ask_before_showing = True, ask_answers = True):
    """
    n: number of questions to be asked
    ask_unanswered: ask only questions without entry in answer_dict
    show_all_questions_first: print all drawn questions
    show_answers: include answers when printing drawn questions (will only show if show_all_questions_first = True)
    ask_before_showing: require an keyboard action before showing the answer to a question
    ask_answers: ask answers to questions. Inputs will be included in answer dict (which will be included in anki cards)
    """
    total_n_q = len([q for tut in nested_list for q in tut])
    if n> total_n_q:
        ValueError(f"n is too large (more than {total_n_q})")       

    valid_type = ["unanswered", "answered","all"] 
    if question_type not in valid_type:
        raise ValueError(f"Invalid value for question_type. Allowed values are {valid_type}.")
    
    # get all questions in dict form
    question_pool = {index + 1: sublist for index, sublist in enumerate(nested_list)}

    #initialize empty objects
    questions = []
    q_correct_type = {}

    # find questions of requested type
    if question_type != "all":
        if question_type == "unanswered":
            for tut_num_1, tut in enumerate(nested_list):
                tut_num = tut_num_1 + 1 
                if tut_num not in answer_dict.keys():
                    q_correct_type[tut_num] = tut
                else:
                    q_correct_type[tut_num] = [q for q in tut if get_question_num(q) not in answer_dict[tut_num].keys()]
        elif question_type == "answered":
            for tut_num_1, tut in enumerate(nested_list):
                tut_num = tut_num_1 + 1 
                if tut_num in answer_dict.keys():
                    q_correct_type[tut_num] = [q for q in tut if get_question_num(q) in answer_dict[tut_num].keys()]
                    
        #if there are less unanswered questions than requested questions
        count_q_type = len([q for tut in q_correct_type.keys() for q in q_correct_type[tut]])
        if(count_q_type < n):
            print(f"Only {count_q_type} questions of selected type available. filled up with other questions")
            n_add = n - count_q_type
            other_questions = random.sample([f"{str(tut)} {q}" for tut in question_pool.keys() for q in question_pool[tut]], n_add)
            questions = random.sample([f"{str(tut)} {q}" for tut in q_correct_type.keys() for q in q_correct_type[tut]], count_q_type)
            #ask unanswered first (random order) and fill up with other questions
            questions.extend(other_questions)        
        else:
            question_pool = q_correct_type  
    
    #if the random questions werent chosen yet (due to too little unanswered questions)
    if len(questions) < 1:
       #choose at random
       questions = random.sample([f"{str(tut)} {q}" for tut in question_pool.keys() for q in question_pool[tut]], n)

    if show_all_questions_first:
        for q in questions:
            print("Tut "+ q)
            if show_answers:
                if ask_before_showing:
                    input("press enter or escape to show answer")
                split_question = split_tut_question_text(q)
                print("Answer: " + find_answer(f"{split_question[0][1]} {split_question[0][2]}", int(split_question[0][0])))      

    if ask_answers:
        name = input("Under which name do you want to answer the questions?" + "\n")
        if len(name) < 1:
            name = "x"
        # List to store answers
        answers = []

        # Ask each question and store the answers
        for question in questions:
            tut_question_text = split_tut_question_text(question)
            answer = ask_question(question)
            if len(answer)>0:
                print(question)
                print("Your answer:" + answer)
                answer = f"{tut_question_text[0][0]} {tut_question_text[0][1]} {answer}"
                answers.append(answer)


        # Convert the list of answers to lines in a .txt document
        filename = f"test your understanding answers {name}.txt"

        # Check if the file exists
        if os.path.exists(filename):
            # Append content to an existing file
            with open(filename, "a") as file:
                for answer in answers:
                    file.write(f"{answer}\n")
        else:
            # Create a new file
            with open(filename, "w") as file:
                for answer in answers:
                    file.write(f"{answer}\n")

        #put the answers into the answer_dict            
        append_to_answer_dict(answers, name)

In [5]:
#Draw random questions
print("#Show 1 question (without the answer)")
test(n = 1,show_all_questions_first= True, show_answers = False, ask_answers=False) 
print("#Show 2 answered questions, ask before showing the second answer")
test(n = 1,question_type="answered", show_all_questions_first= True, ask_before_showing= False, ask_answers=False) 
test(n = 1,question_type="answered", show_all_questions_first= True, ask_answers=False) 
print("#Show 1 unanswered question")
test(n = 1,question_type="unanswered", show_all_questions_first= True, ask_before_showing= False, ask_answers=False) 
print("#Now you answer 2 unanswered questions, if you don't enter a name, pseudonym x will be assumed")
test(n = 2) 

#Show 1 question (without the answer)
Tut 9 i) Describe and contrast L1VM and SVM for classification. In both cases,
how is sparsity achieved? Which hyperparameters exist and what do they
affect?
#Show 2 answered questions, ask before showing the second answer
Tut 1 d) Which distributions do you know? Describe these distributions and their
properties.
Answer: leon- Discrete distributions: Bernulli (Head or tail, p(1) = theta), Binomial  (how many heads in n trows, Binom(n_h,n, theta) = \binom{n}{n_h} \theta^{n_H}* (1-\theta)^{n-n_h}), Categorical (which catagory p(k)= \theta_k), Multinomial (how often which distribution in n trials Mu(n|n,theta)= \binom{n}{n_1 ... n_K} \prod^K_{k=1} \theta^{n_k}_k); Continous distributions:  gaussian, Beta, Dirichlet, multivariate gaussian
Tut 8 j) Lloyd’s algorithm for K-means clustering is related to a certain variant of
the EM algorithm and a certain variant of GMMs. Which variants? What
would happen if we use this variant of the EM algorithm with a

In [ ]:
#answer_dict = {} # if you don' want to include any answers, just uncomment this line 
deck_name = "ML 2023 Test your understanding"  # Change this to your deck name (this deck will be created if it doesn't exist allready)
anki_url = "http://localhost:8765"  # AnkiConnect default URL

#put all questions and Answers (if exist) into Anki list
anki_list = [{"note_type": "Basic",
               "fields": {"Front":f"Tut {tut_i+1}\n{q}",
                          "Back": find_answer(q, tut_i+1)},
                "tags": [f"Tut_{tut_i+1}"]} for tut_i, tut in enumerate(nested_list) for q in tut]

#check if deck exists, create if not
def deck_exists(deck_name):
    # Build the AnkiConnect API request payload for deck check
    payload = {
        "action": "deckNames",
        "version": 6,
    }

    # Make the API request to AnkiConnect to get the list of existing deck names
    response = requests.post(anki_url, json=payload)

    # Check the response status
    if response.status_code == 200:
        existing_decks = response.json()["result"]
        return deck_name in existing_decks
    else:
        print(f"An error occurred while checking for existing decks: {response.text}")
        return False
def create_deck(deck_name):
    # Build the AnkiConnect API request payload for creating a deck
    payload = {
        "action": "createDeck",
        "version": 6,
        "params": {
            "deck": deck_name,
        }
    }

    # Make the API request to AnkiConnect to create the deck
    response = requests.post(anki_url, json=payload)

    # Check the response status
    if response.status_code == 200:
        print(f"Deck '{deck_name}' created successfully.")
    else:
        print(f"An error occurred while creating the deck: {response.text}")


if not deck_exists(deck_name):
    # Create the deck if it doesn't exist
    create_deck(deck_name)
else:
    print(f"Deck '{deck_name}' already exists.")
    if "Y" != input("Type 'Y' if you want to create new cards in deck: " + deck_name):
        raise ValueError("Didn't type 'Y'")

#create cards
def create_anki_card(payload):
    response = requests.post(anki_url, json=payload)
    return response.json()

def create_anki_cards(cards_data):
    with ThreadPoolExecutor() as executor:
        payloads = []
        for card in cards_data:
            tags = ["ML", "Data_Science"]
            note_type = card["note_type"]
            fields = card["fields"]
            tags.extend(card["tags"])

            # Construct the payload for creating a new note
            payload = {
                "action": "addNote",
                "version": 6,
                "params": {
                    "note": {
                        "deckName": deck_name,
                        "modelName": note_type,
                        "fields": fields,
                        "tags": tags,  # Add tags if needed
                    }
                }
            }
            payloads.append(payload)
        # Use ThreadPoolExecutor to parallelize the requests
        responses = list(executor.map(create_anki_card, payloads))

        # Print the responses (optional)
        for response in responses:
            print(response)

create_anki_cards(anki_list) #Took 35sec on my machine